In [ ]:
# Load libraries
import pandas as pd
import numpy as np
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
#import seaborn as sb



from sklearn import metrics, cross_validation, linear_model, ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import neural_network
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline
#afficher les graphique ici dans le navigateur

import json
import codecs
from random import randint

In [ ]:
TestData=pd.read_csv('test_imp_features.csv',index_col=0)
X_train=pd.read_csv('datas/train_final.csv',index_col=0)
X_valid=pd.read_csv('datas/valid_final.csv',index_col=0)
X_hold=pd.read_csv('datas/hold_final.csv',index_col=0)
Y_train,Y_valid,Y_hold=X_train.TARGET,X_valid.TARGET,X_hold.TARGET
X_train,X_valid,X_hold=X_train.drop("TARGET",axis=1),X_valid.drop("TARGET",axis=1),X_hold.drop("TARGET",axis=1)

In [ ]:
imporatnce_features=['var38',
                     'var15',
                     'saldo_medio_var5_hace3',
                     'saldo_medio_var5_ult3',
                     'num_var45_ult3',
                     'num_var45_hace3',
                     'num_var45_hace2',
                     'num_var22_ult3',
                     'saldo_medio_var5_hace2',
                     'num_var45_ult1',
                     'num_med_var45_ult3',
                     'num_var22_hace3',
                     'saldo_var42',
                     'num_var22_hace2',
                     'saldo_medio_var5_ult1',
                     'saldo_var30',
                     'saldo_var5',
                     'num_meses_var5_ult3',
                     'var36',
                     'num_meses_var39_vig_ult3',
                     'num_var22_ult1',
                     'ind_var30',
                     'num_med_var22_ult3',
                     'num_var42',
                     'imp_op_var39_ult1',
                     'imp_op_var39_efect_ult3',
                     'imp_ent_var16_ult1',
                     'imp_op_var41_ult1',
                     'var3',
                     'imp_trans_var37_ult1',
                     ]
len(imporatnce_features)

In [ ]:
RAND_STATE=7
scoring = 'roc_auc'
N_TREES = 500

PARAM_GRID = {
    'LR': {'C': [10**x for x in range(-8,5)],
           'class_weight': ['balanced'],
           'penalty':['l2'],
           'solver' :['newton-cg', 'lbfgs', 'liblinear', 'sag'],
           'random_state':[RAND_STATE,],
    },
    
    'RFC': {
        'n_jobs': [3],
        'criterion':["gini","entropy"],
        "max_features":["sqrt",None],
        "max_depth":[180,200,None],
        "n_estimators":[500,1000],
        "min_samples_split":[2,5,10],
        "class_weight": ['balanced','balanced_subsample',None],
        'random_state':[RAND_STATE,],
    },
    'DTC': {#DecisionTreeClassifier
        "criterion":["gini","entropy"],
        "splitter":["best","random"],
        "max_features":["sqrt",None],
        "max_depth":[None,],
        "min_samples_split":[2,5,9,12,15],
        'class_weight': ['balanced',None],
        "presort":[False],
        'random_state':[RAND_STATE,],
    },
    'ExtraTreesClassifier': {'min_samples_leaf': [2, 3],#copie de ubuntu rush
                             'n_jobs': [5],
                             'min_samples_split': [1, 2, 5],
                             'bootstrap': [False],
                             'max_depth': [15, 20, 25, 30],
                             'max_features': [1, 3, 5, 8],
                             'random_state':[RAND_STATE,],
    },
    'GBC': {'max_features': [4, 5, 6, 7, 8],
            'learning_rate': [.05, .08, .1],
            'max_depth': [8, 10, 13],
            'random_state':[RAND_STATE,],
    },
    'SVC':{'C':[10**x for x in range(-7,4)],
           'kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
           'gamma':['auto',0.5,0.7],
           'degree':[2,3],
           'class_weight': ['balanced',],
           'probability':[True,],
           'random_state':[RAND_STATE,],
          },
    'KNN':{ 'n_neighbors' : list({randint(1,500) for i in range(8)}),
            'weights' : ['uniform','distance'],
            'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'leaf_size' : [20,30,40,60],
            'metric' : ['minkowski',],
            #'n_jobs' :[2,],
            'p' : [1,2,7,10],
          },
    'NNC':{'solver' : ['lbfgs', 'sgd', 'adam'],
           'hidden_layer_sizes':[1,2,4,8,32,128],
           'activation':['identity', 'logistic', 'tanh', 'relu'],
           'alpha':[0.0001,0.005],#L2 penalty (regularization term) parameter.
           'batch_size':['auto'],#auto=min(200,n_samples)
           'learning_rate': ['constant', 'invscaling', 'adaptive'],
           #'momentu':[0,0.2,0.5,0.9], 
           'random_state':[RAND_STATE,],
    },
    'BAGC':{#BaggingClassifier
        'base_estimator':[DecisionTreeClassifier(),linear_model.LogisticRegression(),ensemble.GradientBoostingClassifier()],#,KNeighborsClassifier(),],
        'n_estimators':[10,20,100],
        'max_samples':[0.6,0.8],
        'max_features':[0.6,0.9],
        'n_jobs':[2,],
        'random_state':[RAND_STATE,],
        
    }
}

models= {
    'LR': linear_model.LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    #'GBC': ensemble.GradientBoostingClassifier(),
    'RFC': ensemble.RandomForestClassifier(),
    #'ExtraTreesClassifier': ensemble.ExtraTreesClassifier(),
    'SVC':SVC(),
    'NNC': neural_network.MLPClassifier(),
    'DTC': DecisionTreeClassifier(),
    'BAGC': ensemble.BaggingClassifier(),
    
}

In [ ]:

def generer20Feature():
    nb=len(imporatnce_features)
    a=set()
    while len(a)<=20:
        a.add(randint(0,nb-1))
    return list(a)

In [ ]:
def generatorParameters(algoName):
    params=PARAM_GRID[algoName]
    prms=dict()
    ch="""def genarator_parameters(nameAlgo):
    params=PARAM_GRID[nameAlgo]
    prms=dict()
    
    """
    i=0
    for key,val in params.items():
        ch+='\t'*i
        ch+="""for """+key+""" in params['"""+key+"""']:
        """+('\t'*(i+1))+"""prms['"""+key+"""']="""+key+"""
            """
        i+=1
    ch+='\t'*i
    ch+="""yield prms
    """
    ch+='\t'*(i+1)
    #ch+="""prms=dict()"""
    exec(ch)
    return genarator_parameters(algoName)
    
for pr in generatorParameters('GBC'):
    print pr
        
 

In [ ]:
def execAlgo(ALGO,set_features,parameters,file_json,i,prefix_id="",score_seuil=0.5):
    score=0
    try:
        ALGO.set_params(**parameters)
        ALGO.fit(X_train[set_features],Y_train)
        ALGO_preds = ALGO.predict_proba(X_valid[set_features])[:, 1]
        fpr,tpr,ll=metrics.roc_curve(Y_valid, ALGO_preds)
        score=metrics.auc(fpr, tpr)
        #print score,":",set_features
        if score>=score_seuil:#si >=score_seuil on stock
            params=parameters
            idd=prefix_id+"_"+str(i)
            obj_json=dict()
            obj_json["id"]=idd
            obj_json["score"]=score
            obj_json["features"]=set_features
            obj_json["valid"]=ALGO_preds#ALGO.predict(X_valid[set_features]).tolist()
            obj_json["hold"]=ALGO.predict_proba(X_hold[set_features])[:, 1]#ALGO.predict(X_hold[set_features]).tolist()
            obj_json["test"]=ALGO.predict_proba(TestData[set_features])[:, 1]#ALGO.predict(TestData[set_features]).tolist()
            if type(ALGO)==type(ensemble.BaggingClassifier()):
                #prob de parameter 'base_estimator' is Not Serializable
                #we have been stocked the estimator class name
                params={i:j for i,j in parameters.items() if i!='base_estimator'}
                params['base_estimator']=str(parameters['base_estimator'].__class__).split("'")[1].split(".")[-1]
                print params['base_estimator']," score:",score
            obj_json["params"]=params
            json.dump(obj=obj_json,fp=file_json,indent=5)
            file_json.flush()
            i+=1#rise the number of classifier
    except Exception as e:
        print e
    return i,score#return the next position of next classifier, and score

In [ ]:
def run_stock_for_algo(model_name,thresholds_score=0.6):
    with open("results/"+model_name+".amh","a") as dtOut:#results file
        seuil_score=thresholds_score
        #obj_json=dict()
        i,min_score,max_score=1,1,0
        ALGO=models[model_name]
        all_features=list(range(len(imporatnce_features)))
        for parameters in generatorParameters(model_name):
            j=i
            set_features=generer20Feature()
            i,score=execAlgo(ALGO,all_features,parameters,dtOut,i,model_name,seuil_score)
            min_score=score if score<min_score and i>j else min_score
            max_score=score if score>max_score and i>j else max_score
            j=i
            i,score=execAlgo(ALGO,set_features,parameters,dtOut,i,model_name,seuil_score)
            min_score=score if score<min_score and i>j else min_score
            max_score=score if score>max_score and i>j else max_score
        #json.dump(obj=obj_json,fp=dtOut,indent=5)
        dtOut.flush()
        print i-1, "Classifier :"+model_name,"  with the scores between:",min_score,"and ",max_score